## Esplora il clustering K-Means utilizzando R e i principi dei dati Tidy.

### [**Quiz pre-lezione**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

In questa lezione, imparerai a creare cluster utilizzando il pacchetto Tidymodels e altri pacchetti dell'ecosistema R (li chiameremo amici 🧑‍🤝‍🧑), insieme al dataset di musica nigeriana che hai importato in precedenza. Tratteremo le basi del K-Means per il clustering. Tieni presente che, come hai appreso nella lezione precedente, ci sono molti modi per lavorare con i cluster e il metodo che utilizzi dipende dai tuoi dati. Proveremo il K-Means poiché è la tecnica di clustering più comune. Iniziamo!

Termini che imparerai:

-   Punteggio silhouette

-   Metodo del gomito

-   Inerzia

-   Varianza

### **Introduzione**

[Il clustering K-Means](https://wikipedia.org/wiki/K-means_clustering) è un metodo derivato dal campo dell'elaborazione del segnale. Viene utilizzato per dividere e raggruppare gruppi di dati in `k cluster` basandosi sulle somiglianze delle loro caratteristiche.

I cluster possono essere visualizzati come [diagrammi di Voronoi](https://wikipedia.org/wiki/Voronoi_diagram), che includono un punto (o 'seme') e la sua corrispondente regione.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografica di Jen Looper</figcaption>


Il clustering K-Means segue i seguenti passaggi:

1.  Lo scienziato dei dati inizia specificando il numero desiderato di cluster da creare.

2.  Successivamente, l'algoritmo seleziona casualmente K osservazioni dal dataset per fungere da centri iniziali per i cluster (cioè, i centroidi).

3.  Ogni osservazione rimanente viene quindi assegnata al suo centroide più vicino.

4.  Successivamente, vengono calcolate le nuove medie di ciascun cluster e il centroide viene spostato verso la media.

5.  Ora che i centri sono stati ricalcolati, ogni osservazione viene nuovamente controllata per verificare se potrebbe essere più vicina a un cluster diverso. Tutti gli oggetti vengono riassegnati utilizzando le medie dei cluster aggiornate. I passaggi di assegnazione dei cluster e aggiornamento dei centroidi vengono ripetuti iterativamente fino a quando le assegnazioni dei cluster smettono di cambiare (cioè, quando si raggiunge la convergenza). Tipicamente, l'algoritmo termina quando ogni nuova iterazione comporta uno spostamento trascurabile dei centroidi e i cluster diventano statici.

<div>

> Nota che, a causa della randomizzazione delle k osservazioni iniziali utilizzate come centroidi di partenza, possiamo ottenere risultati leggermente diversi ogni volta che applichiamo la procedura. Per questo motivo, la maggior parte degli algoritmi utilizza diversi *avvii casuali* e sceglie l'iterazione con il WCSS più basso. Pertanto, è fortemente consigliato eseguire sempre il K-Means con diversi valori di *nstart* per evitare un *ottimo locale indesiderato.*

</div>

Questa breve animazione, utilizzando le [illustrazioni](https://github.com/allisonhorst/stats-illustrations) di Allison Horst, spiega il processo di clustering:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Illustrazione di @allison_horst</figcaption>



Una domanda fondamentale che sorge nel clustering è questa: come si fa a sapere in quanti cluster separare i dati? Uno svantaggio dell'utilizzo del K-Means è che sarà necessario stabilire `k`, ovvero il numero di `centroidi`. Fortunatamente, il `metodo del gomito` aiuta a stimare un buon valore iniziale per `k`. Lo proverai tra poco.

### 

**Prerequisiti**

Riprenderemo esattamente da dove ci siamo fermati nella [lezione precedente](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), dove abbiamo analizzato il dataset, creato molte visualizzazioni e filtrato il dataset per osservazioni di interesse. Assicurati di dare un'occhiata!

Avremo bisogno di alcuni pacchetti per completare questo modulo. Puoi installarli con: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

In alternativa, lo script qui sotto verifica se hai i pacchetti necessari per completare questo modulo e li installa per te nel caso in cui ne manchino alcuni.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


## 1. Un ballo con i dati: Restringi ai 3 generi musicali più popolari

Questo è un riepilogo di ciò che abbiamo fatto nella lezione precedente. Analizziamo un po' di dati!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 È andata bene!

## 2. Ulteriore esplorazione dei dati.

Quanto sono puliti questi dati? Controlliamo la presenza di valori anomali utilizzando i box plot. Ci concentreremo sulle colonne numeriche con meno valori anomali (anche se potresti eliminare i valori anomali). I box plot possono mostrare l'intervallo dei dati e aiutare a scegliere quali colonne utilizzare. Nota, i box plot non mostrano la varianza, un elemento importante per dati ben clusterizzabili. Consulta [questa discussione](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) per ulteriori approfondimenti.

I [box plot](https://en.wikipedia.org/wiki/Box_plot) vengono utilizzati per rappresentare graficamente la distribuzione dei dati `numerici`, quindi iniziamo *selezionando* tutte le colonne numeriche insieme ai generi musicali più popolari.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Vedi come il selettore `where` rende tutto più semplice 💁? Esplora altre funzioni simili [qui](https://tidyselect.r-lib.org/).

Poiché creeremo un boxplot per ogni caratteristica numerica e vogliamo evitare di usare cicli, riformattiamo i nostri dati in un formato *più lungo* che ci permetterà di sfruttare i `facets` - sottotrame che mostrano ciascuna un sottoinsieme dei dati.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Molto più lungo! Ora è il momento di alcuni `ggplots`! Quindi quale `geom` useremo?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Facile-gg!

Ora possiamo vedere che questi dati sono un po' rumorosi: osservando ogni colonna come un boxplot, si possono notare dei valori anomali. Potresti esaminare il dataset e rimuovere questi valori anomali, ma ciò renderebbe i dati piuttosto ridotti.

Per il momento, scegliamo quali colonne utilizzare per il nostro esercizio di clustering. Selezioniamo le colonne numeriche con intervalli simili. Potremmo codificare `artist_top_genre` come numerico, ma per ora lo escluderemo.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Calcolo del clustering k-means in R

Possiamo calcolare il k-means in R utilizzando la funzione integrata `kmeans`, consulta `help("kmeans()")`. La funzione `kmeans()` accetta un data frame con tutte le colonne numeriche come suo argomento principale.

Il primo passo nell'utilizzo del clustering k-means è specificare il numero di cluster (k) che verranno generati nella soluzione finale. Sappiamo che ci sono 3 generi musicali che abbiamo estratto dal dataset, quindi proviamo con 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


L'oggetto kmeans contiene diverse informazioni ben spiegate in `help("kmeans()")`. Per ora, concentriamoci su alcune di esse. Vediamo che i dati sono stati raggruppati in 3 cluster di dimensioni 65, 110, 111. L'output contiene anche i centri dei cluster (medie) per i 3 gruppi rispetto alle 5 variabili.

Il vettore di clustering rappresenta l'assegnazione del cluster per ogni osservazione. Utilizziamo la funzione `augment` per aggiungere l'assegnazione del cluster al set di dati originale.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfetto, abbiamo appena suddiviso il nostro set di dati in un insieme di 3 gruppi. Quindi, quanto è buono il nostro clustering 🤷? Diamo un'occhiata al `Silhouette score`.

### **Silhouette score**

[L'analisi del silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) può essere utilizzata per studiare la distanza di separazione tra i cluster risultanti. Questo punteggio varia da -1 a 1, e se il punteggio è vicino a 1, il cluster è denso e ben separato dagli altri cluster. Un valore vicino a 0 rappresenta cluster sovrapposti con campioni molto vicini al confine decisionale dei cluster vicini. [fonte](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Il metodo della silhouette media calcola la silhouette media delle osservazioni per diversi valori di *k*. Un punteggio medio di silhouette alto indica un buon clustering.

La funzione `silhouette` nel pacchetto cluster serve per calcolare la larghezza media della silhouette.

> La silhouette può essere calcolata con qualsiasi [metrica di distanza](https://en.wikipedia.org/wiki/Distance "Distance"), come la [distanza euclidea](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") o la [distanza di Manhattan](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") di cui abbiamo discusso nella [lezione precedente](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Il nostro punteggio è **.549**, quindi esattamente a metà. Questo indica che i nostri dati non sono particolarmente adatti a questo tipo di clustering. Vediamo se possiamo confermare questa ipotesi visivamente. Il [pacchetto factoextra](https://rpkgs.datanovia.com/factoextra/index.html) fornisce funzioni (`fviz_cluster()`) per visualizzare il clustering.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


La sovrapposizione nei cluster indica che i nostri dati non sono particolarmente adatti a questo tipo di clustering, ma continuiamo comunque.

## 4. Determinare il numero ottimale di cluster

Una domanda fondamentale che spesso emerge nel clustering K-Means è questa: senza etichette di classe conosciute, come si può sapere in quanti cluster suddividere i dati?

Un modo per scoprirlo è utilizzare un campione di dati per `creare una serie di modelli di clustering` con un numero crescente di cluster (ad esempio da 1 a 10) e valutare metriche di clustering come il **Silhouette score.**

Determiniamo il numero ottimale di cluster calcolando l'algoritmo di clustering per diversi valori di *k* e valutando il **Within Cluster Sum of Squares** (WCSS). La somma totale dei quadrati all'interno del cluster (WCSS) misura la compattezza del clustering e vogliamo che sia il più piccolo possibile, poiché valori più bassi indicano che i punti dati sono più vicini tra loro.

Esploriamo l'effetto di diverse scelte di `k`, da 1 a 10, su questo clustering.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Ora che abbiamo il totale della somma dei quadrati intra-cluster (tot.withinss) per ciascun algoritmo di clustering con centro *k*, utilizziamo il [metodo del gomito](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) per trovare il numero ottimale di cluster. Il metodo consiste nel tracciare il WCSS in funzione del numero di cluster e scegliere il [gomito della curva](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") come il numero di cluster da utilizzare.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Il grafico mostra una grande riduzione in WCSS (quindi maggiore *compattezza*) man mano che il numero di cluster aumenta da uno a due, e una ulteriore riduzione evidente da due a tre cluster. Dopo di ciò, la riduzione è meno marcata, risultando in un `gomito` 💪 nel grafico intorno a tre cluster. Questo è un buon indicatore che ci sono due o tre cluster di punti dati ragionevolmente ben separati.

Possiamo ora procedere ed estrarre il modello di clustering dove `k = 3`:

> `pull()`: utilizzato per estrarre una singola colonna
>
> `pluck()`: utilizzato per indicizzare strutture dati come liste


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Fantastico! Procediamo a visualizzare i cluster ottenuti. Ti interessa un po' di interattività utilizzando `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Forse ci saremmo aspettati che ogni cluster (rappresentato da colori diversi) avesse generi distinti (rappresentati da forme diverse).

Diamo un'occhiata alla precisione del modello.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


La precisione di questo modello non è male, ma nemmeno eccezionale. Potrebbe essere che i dati non si prestino bene al clustering con K-Means. Questi dati sono troppo sbilanciati, poco correlati e presentano troppa varianza tra i valori delle colonne per essere clusterizzati in modo efficace. Infatti, i cluster che si formano sono probabilmente fortemente influenzati o distorti dalle tre categorie di genere che abbiamo definito sopra.

Tuttavia, è stato un processo di apprendimento interessante!

Nella documentazione di Scikit-learn, si può vedere che un modello come questo, con cluster non molto ben definiti, presenta un problema di "varianza":

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografica da Scikit-learn</figcaption>



## **Varianza**

La varianza è definita come "la media delle differenze al quadrato rispetto alla media" [fonte](https://www.mathsisfun.com/data/standard-deviation.html). Nel contesto di questo problema di clustering, si riferisce al fatto che i numeri del nostro dataset tendono a divergere un po' troppo dalla media.

✅ Questo è un ottimo momento per pensare a tutti i modi in cui potresti correggere questo problema. Modificare un po' i dati? Usare colonne diverse? Utilizzare un algoritmo differente? Suggerimento: prova a [scalare i tuoi dati](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) per normalizzarli e testare altre colonne.

> Prova questo '[calcolatore di varianza](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' per comprendere meglio il concetto.

------------------------------------------------------------------------

## **🚀Sfida**

Dedica un po' di tempo a questo notebook, modificando i parametri. Riesci a migliorare la precisione del modello pulendo ulteriormente i dati (ad esempio, rimuovendo i valori anomali)? Puoi utilizzare pesi per dare maggiore importanza a determinati campioni di dati. Cos'altro puoi fare per creare cluster migliori?

Suggerimento: prova a scalare i tuoi dati. Nel notebook c'è del codice commentato che aggiunge una scalatura standard per far sì che le colonne dei dati si somiglino di più in termini di intervallo. Noterai che, mentre il punteggio silhouette diminuisce, il "gomito" nel grafico del gomito si appiana. Questo accade perché lasciare i dati non scalati permette ai dati con minore varianza di avere un peso maggiore. Leggi di più su questo problema [qui](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Quiz post-lezione**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Revisione e Studio Autonomo**

-   Dai un'occhiata a un simulatore di K-Means [come questo](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Puoi utilizzare questo strumento per visualizzare punti dati di esempio e determinare i loro centroidi. Puoi modificare la casualità dei dati, il numero di cluster e il numero di centroidi. Questo ti aiuta a farti un'idea di come i dati possono essere raggruppati?

-   Consulta anche [questo documento su K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) di Stanford.

Vuoi mettere alla prova le tue nuove competenze di clustering con dataset che si prestano bene al clustering con K-Means? Consulta:

-   [Addestrare e Valutare Modelli di Clustering](https://rpubs.com/eR_ic/clustering) utilizzando Tidymodels e altri strumenti

-   [Analisi dei Cluster con K-Means](https://uc-r.github.io/kmeans_clustering), Guida alla Programmazione R per l'Analisi Aziendale dell'UC

- [Clustering con K-Means e principi di dati ordinati](https://www.tidymodels.org/learn/statistics/k-means/)

## **Compito**

[Prova metodi di clustering diversi](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## GRAZIE A:

[Jen Looper](https://www.twitter.com/jenlooper) per aver creato la versione originale in Python di questo modulo ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) per aver creato le incredibili illustrazioni che rendono R più accogliente e coinvolgente. Trova altre illustrazioni nella sua [galleria](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Buono studio,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Illustrazione di @allison_horst</figcaption>



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche potrebbero contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un esperto umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
